### Model Optimizasyonu ve Overfitting Yönetimi
Selin Hoca'nın slaytındaki 'Overfitting' riskini yönetmek ve modelin genelleme yeteneğini artırmak için Optuna ile hiperparametre optimizasyonu yaptım.
Validation setini kullanarak `learning_rate`, `num_leaves` ve regülasyon (`lambda`) parametrelerini bilimsel olarak ayarladım.

In [1]:
import pandas as pd
import numpy as np
import optuna

In [2]:
train_df = pd.read_csv('../data/ranking/train.csv')
val_df = pd.read_csv('../data/ranking/val.csv')

In [3]:
q_train = train_df.groupby('Event_Block', sort=False).size().to_list()
q_val = val_df.groupby('Event_Block', sort=False).size().to_list()

In [4]:
drop_cols = ['Class', 'Time', 'Event_Block']

In [5]:
X_train = train_df.drop(drop_cols, axis=1)
y_train = train_df['Class']

In [6]:
X_val = val_df.drop(drop_cols, axis=1)
y_val = val_df['Class']

In [7]:
print(f"Train Gruplari: {len(q_train)} adet")
print(f"Val Gruplari: {len(q_val)} adet")

Train Gruplari: 33 adet
Val Gruplari: 7 adet


In [11]:
import lightgbm as lgb
from sklearn.metrics import ndcg_score

In [12]:
def objective(trial):
    params = {
        'objective': 'lambdarank', 
        'metric': 'ndcg',          
        'eval_at': 5,              
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'random_state': 42,
        
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'lambda_l1': trial.suggest_float('lambda_l1', 0, 10),
        'lambda_l2': trial.suggest_float('lambda_l2', 0, 10),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
    }

    model = lgb.LGBMRanker(**params)

    model.fit(
        X_train, y_train,
        group=q_train,
        eval_set=[(X_val, y_val)],
        eval_group=[q_val],
        callbacks=[lgb.early_stopping(stopping_rounds=50)] 
    )

    score = model.best_score_['valid_0']['ndcg@5']
    return score

print("Optimizasyon başlıyor... (Bu işlem birkaç dakika sürebilir)")
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

print("\n------------------------------------------------")
print("EN İYİ PARAMETRELER:", study.best_params)
print("EN İYİ SKOR (NDCG@5):", study.best_value)
print("------------------------------------------------")

[I 2025-11-30 18:12:52,240] A new study created in memory with name: no-name-bee0d369-a2ca-4ecf-8db9-a0864f808afe


Optimizasyon başlıyor... (Bu işlem birkaç dakika sürebilir)
Training until validation scores don't improve for 50 rounds


/Users/pinar/fraud-detection-bootcamp/venv/lib/python3.13/site-packages/lightgbm/sklearn.py:861: UserWarning: Found 'eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")
[I 2025-11-30 18:12:52,831] Trial 0 finished with value: 0.9603895103778503 and parameters: {'learning_rate': 0.04792845379393924, 'num_leaves': 112, 'lambda_l1': 8.707290590061314, 'lambda_l2': 8.518315023335091, 'bagging_fraction': 0.6913322652438483, 'feature_fraction': 0.8900113205710999}. Best is trial 0 with value: 0.9603895103778503.


Did not meet early stopping. Best iteration is:
[96]	valid_0's ndcg@5: 0.96039
Training until validation scores don't improve for 50 rounds


/Users/pinar/fraud-detection-bootcamp/venv/lib/python3.13/site-packages/lightgbm/sklearn.py:861: UserWarning: Found 'eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")
[I 2025-11-30 18:12:53,445] Trial 1 finished with value: 0.9603895103778503 and parameters: {'learning_rate': 0.0442149038843821, 'num_leaves': 73, 'lambda_l1': 9.997749019978025, 'lambda_l2': 5.960911384846318, 'bagging_fraction': 0.6331726462735414, 'feature_fraction': 0.7624139626863615}. Best is trial 0 with value: 0.9603895103778503.


...

Early stopping, best iteration is:
[45]	valid_0's ndcg@5: 0.981256
Training until validation scores don't improve for 50 rounds


/Users/pinar/fraud-detection-bootcamp/venv/lib/python3.13/site-packages/lightgbm/sklearn.py:861: UserWarning: Found 'eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")
[I 2025-11-30 18:13:05,618] Trial 18 finished with value: 0.9603895103778503 and parameters: {'learning_rate': 0.09357110195935393, 'num_leaves': 64, 'lambda_l1': 5.903010360328257, 'lambda_l2': 3.347626816320904, 'bagging_fraction': 0.5038311234770345, 'feature_fraction': 0.6962217575875714}. Best is trial 2 with value: 1.0.


Early stopping, best iteration is:
[43]	valid_0's ndcg@5: 0.96039
Training until validation scores don't improve for 50 rounds


/Users/pinar/fraud-detection-bootcamp/venv/lib/python3.13/site-packages/lightgbm/sklearn.py:861: UserWarning: Found 'eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")
[I 2025-11-30 18:13:06,605] Trial 19 finished with value: 1.0 and parameters: {'learning_rate': 0.19115677497326883, 'num_leaves': 112, 'lambda_l1': 1.2158755007973108, 'lambda_l2': 9.940314941065147, 'bagging_fraction': 0.7252335694073534, 'feature_fraction': 0.9472093216417008}. Best is trial 2 with value: 1.0.


Did not meet early stopping. Best iteration is:
[66]	valid_0's ndcg@5: 1

------------------------------------------------
EN İYİ PARAMETRELER: {'learning_rate': 0.0995788010026312, 'num_leaves': 24, 'lambda_l1': 0.9057918177073165, 'lambda_l2': 1.1653184276747852, 'bagging_fraction': 0.8420171866482506, 'feature_fraction': 0.6296344145026685}
EN İYİ SKOR (NDCG@5): 1.0
------------------------------------------------


In [14]:
best_params = study.best_params

In [15]:
best_params['objective'] = 'lambdarank'
best_params['metric'] = 'ndcg'
best_params['eval_at'] = 5
best_params['random_state'] = 42

In [16]:
final_model = lgb.LGBMRanker(**best_params)

In [17]:
final_model.fit(
    X_train, y_train,
    group=q_train,
    eval_set=[(X_val, y_val)],
    eval_group=[q_val],
    callbacks=[lgb.early_stopping(stopping_rounds=50)]
)

/Users/pinar/fraud-detection-bootcamp/venv/lib/python3.13/site-packages/lightgbm/sklearn.py:861: UserWarning: Found 'eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[47]	valid_0's ndcg@5: 1


,boosting_type,'gbdt'
,num_leaves,24
,max_depth,-1
,learning_rate,0.0995788010026312
,n_estimators,100
,subsample_for_bin,200000
,objective,'lambdarank'
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [18]:
import os
import joblib

In [19]:
os.makedirs('../models', exist_ok=True)
joblib.dump(final_model, '../models/lgbm_ranker.pkl')

['../models/lgbm_ranker.pkl']